In [1]:

import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
from io import StringIO, BytesIO

# Add the parent directory to the system path to allow importing from dat-ingestion_bridge
sys.path.insert(0, str(Path.cwd().parent))

# Explicitly import from the correct path
from src.ingestion_utils import process_gym_data, IngestionMetadata

# Create a temporary directory for test files
if not os.path.exists("temp_data"):
    os.makedirs("temp_data")

print("Setup complete. Ready to demonstrate data ingestion and normalization.")


Setup complete. Ready to demonstrate data ingestion and normalization.


### Possible Data Formats

In [2]:
# Scenario 1: CSV with inconsistent delimiters and encoding issues
csv_content_bad_delimiter = """
id;name;age;start_date
1;Alice;30;2022-01-15
2,Bob,24,16/03/2021
3;Charlie;35;2020-11-01
4,David,29,05-07-2023
"""
csv_path_bad_delimiter = "temp_data/bad_delimiter_data.csv"

with open(csv_path_bad_delimiter, "w", encoding="latin1") as f:
    f.write(csv_content_bad_delimiter)


# Scenario 2
excel_path_junk_empty = "temp_data/junk_empty_data.xlsx"
data_excel = {
    'MemberID': ['M001', 'M002', 'M003', 'M004', 'M005', None, 'M007'],
    'Name': ['Eve', 'Frank', 'Grace', None, 'Heidi', 'Ivy', 'Jack'],
    'MembershipType': ['Gold', 'Silver', 'Gold', 'Bronze', 'Silver', 'Gold', 'Bronze'],
    'LastVisit': ['2023-01-10', '15/02/2023', '2023-03-20', 'invalid-date', '2023-05-01', None, '2023-06-11'],
    'Fee': [100.50, 75, 100.50, 50, 75, 100.50, 50],
    'IsActive': ['YES', 'No', '1', '0', 'true', 'FALSE', None],
    'Junk1': [None, None, None, None, None, None, None], # Empty column
    'Junk2': ['garbage', 'garbage', 'garbage', 'garbage', 'garbage', 'garbage', 'garbage']
}
df_excel_raw = pd.DataFrame(data_excel)
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message="The behavior of DataFrame concatenation with empty or all-NA entries is deprecated", category=FutureWarning)
    df_excel_raw.loc[len(df_excel_raw)] = [None] * len(df_excel_raw.columns)

with pd.ExcelWriter(excel_path_junk_empty, engine='openpyxl') as writer:
    df_excel_raw.to_excel(writer, index=False, sheet_name='Sheet1')


# Scenario 3: TSV with long format and missing values
tsv_content_long_missing = """
member_id\tattribute\tvalue
101\tgender\tMale
101\tage\t28
102\tgender\tFemale
102\tage\t34
103\tgender\tNone
103\tage\t30
104\tgender\tMale
104\tage\t
105\tgender\tFemale
105\tage\t22
"""
tsv_path_long_missing = "temp_data/long_missing_data.tsv"
with open(tsv_path_long_missing, "w", encoding="utf-8") as f:
    f.write(tsv_content_long_missing)


# Scenario 4: CSV with mixed types, varying date formats, and partial corruption
csv_content_mixed_corrupt = """
OrderID,Customer,OrderDate,Amount,Status,DeliveryDate
1001,John Doe,2023-01-01,150.75,Completed,2023-01-05
1002,Jane Smith,02/01/2023,abc,Pending,06/01/2023
1003,Peter Jones,2023-Mar-03,200.00,Completed,2023-03-07
1004,Alice Brown,04.04.2023,75.20,Cancelled,invalid-date
1005,Bob White,2023/05/05,120,Completed,2023-May-09
1006,Charlie Green,06-Jun-2023,300.00,Pending,10-Jun-2023
1007,Diana Prince,07/Jul/2023,100,Completed,11/07/2023
1008,Eve Black,invalid-date,50.00,Pending,2023-08-15
"""
csv_path_mixed_corrupt = "temp_data/mixed_corrupt_data.csv"
with open(csv_path_mixed_corrupt, "w", encoding="utf-8") as f:
    f.write(csv_content_mixed_corrupt)

### Scenario 1

In [3]:
# Scenario 1: CSV with inconsistent delimiters and encoding issues

df_processed, metadata = process_gym_data(csv_path_bad_delimiter)

print("\n=== SCENARIO 1: Mixed Delimiters CSV ===")
print(f"\n📊 DataFrame Shape: {df_processed.shape[0]} rows × {df_processed.shape[1]} columns")
print(f"\n🏷️  Columns Identified: {list(df_processed.columns)}")
print(f"\n🎯 Column Roles: {metadata.column_roles}")
print(f"\n📁 File Type: {metadata.file_type}")
print(f"\n📈 Orientation: {metadata.orientation}")

print("\n--- Sample Data ---")
print(df_processed.head())

print("\n--- Data Types ---")
for col, dtype in metadata.inferred_dtypes.items():
    original = metadata.original_dtypes.get(col, 'unknown')
    print(f"  {col}: {original} → {dtype}")

if metadata.warnings:
    print(f"⚠️  Warnings ({len(metadata.warnings)}):")
    for warning in metadata.warnings:
        print(f"   • {warning}")
else:
    print("\n✅ No warnings generated")


=== SCENARIO 1: Mixed Delimiters CSV ===

📊 DataFrame Shape: 4 rows × 4 columns

🏷️  Columns Identified: ['id', 'name', 'age', 'start_date']

🎯 Column Roles: {'id': 'Numeric metric', 'name': 'Categorical dimension', 'age': 'Numeric metric', 'start_date': 'Date/time'}

📁 File Type: csv

📈 Orientation: wide

--- Sample Data ---
   id     name  age  start_date
0   1    Alice   30  2022-01-15
1   2      Bob   24  2021-03-16
2   3  Charlie   35  2020-11-01
3   4    David   29  2023-07-05

--- Data Types ---
  id: object → Int64
  name: object → string
  age: object → Int64
  start_date: object → string
⚠️  Warnings (6):
   • Delimiter ';' with encoding 'utf-8' resulted in inconsistent column counts across rows. Trying other options.
   • Failed to load with delimiter ',' and encoding 'utf-8': Expected 1 fields in line 4, saw 4
   • Attempted to load with delimiter '	' and encoding 'utf-8', but resulted in a single column. Trying other options.
   • Attempted to load with delimiter '|' and 

### Scenario 2

In [4]:
# Scenario 2: Excel file with junk columns, empty rows, and mixed data types

df_processed_excel, metadata_excel = process_gym_data(excel_path_junk_empty)

print("\n=== SCENARIO 2: Excel with Junk Data ===")
print(f"\n📊 DataFrame Shape: {df_processed_excel.shape[0]} rows × {df_processed_excel.shape[1]} columns")
print(f"\n🏷️  Columns Identified: {list(df_processed_excel.columns)}")
print(f"\n🎯 Column Roles: {metadata_excel.column_roles}")
print(f"\n📁 File Type: {metadata_excel.file_type}")
print(f"\n📈 Orientation: {metadata_excel.orientation}")

print("\n--- Sample Data ---")
print(df_processed_excel.head())

print("\n--- Data Types ---")
for col, dtype in metadata_excel.inferred_dtypes.items():
    original = metadata_excel.original_dtypes.get(col, 'unknown')
    print(f"  {col}: {original} → {dtype}")

if metadata_excel.warnings:
    print(f"⚠️  Warnings ({len(metadata_excel.warnings)}):")
    for warning in metadata_excel.warnings:
        print(f"   • {warning}")
else:
    print("\n✅ No warnings generated")


=== SCENARIO 2: Excel with Junk Data ===

📊 DataFrame Shape: 7 rows × 7 columns

🏷️  Columns Identified: ['MemberID', 'Name', 'MembershipType', 'LastVisit', 'Fee', 'IsActive', 'Junk2']

🎯 Column Roles: {'MemberID': 'Categorical dimension', 'Name': 'Categorical dimension', 'MembershipType': 'Categorical dimension', 'LastVisit': 'Other', 'Fee': 'Numeric metric', 'IsActive': 'Numeric metric', 'Junk2': 'Categorical dimension'}

📁 File Type: xlsx

📈 Orientation: long

--- Sample Data ---
  MemberID   Name MembershipType   LastVisit    Fee  IsActive    Junk2
0     M001    Eve           Gold  2023-01-10  100.5      True  garbage
1     M002  Frank         Silver  2023-02-15   75.0     False  garbage
2     M003  Grace           Gold  2023-03-20  100.5      True  garbage
3     M004   <NA>         Bronze         NaN   50.0     False  garbage
4     M005  Heidi         Silver  2023-05-01   75.0      True  garbage

--- Data Types ---
  MemberID: object → string
  Name: object → string
  MembershipT

### Scenario 3

In [5]:
# Scenario 3: TSV with long format and missing values

df_processed_tsv, metadata_tsv = process_gym_data(tsv_path_long_missing)

print("\n=== SCENARIO 3: Long Format TSV ===")
print(f"\n📊 DataFrame Shape: {df_processed_tsv.shape[0]} rows × {df_processed_tsv.shape[1]} columns")
print(f"\n🏷️  Columns Identified: {list(df_processed_tsv.columns)}")
print(f"\n🎯 Column Roles: {metadata_tsv.column_roles}")
print(f"\n📁 File Type: {metadata_tsv.file_type}")
print(f"\n📈 Orientation: {metadata_tsv.orientation}")

print("\n--- Sample Data ---")
print(df_processed_tsv.head())

print("\n--- Data Types ---")
for col, dtype in metadata_tsv.inferred_dtypes.items():
    original = metadata_tsv.original_dtypes.get(col, 'unknown')
    print(f"  {col}: {original} → {dtype}")

if metadata_tsv.warnings:
    print(f"⚠️  Warnings ({len(metadata_tsv.warnings)}):")
    for warning in metadata_tsv.warnings:
        print(f"   • {warning}")
else:
    print("\n✅ No warnings generated")


=== SCENARIO 3: Long Format TSV ===

📊 DataFrame Shape: 10 rows × 3 columns

🏷️  Columns Identified: ['member_id', 'attribute', 'value']

🎯 Column Roles: {'member_id': 'Numeric metric', 'attribute': 'Categorical dimension', 'value': 'Categorical dimension'}

📁 File Type: tsv

📈 Orientation: long

--- Sample Data ---
   member_id attribute   value
0        101    gender    Male
1        101       age      28
2        102    gender  Female
3        102       age      34
4        103    gender    <NA>

--- Data Types ---
  member_id: int64 → Int64
  attribute: object → category
  value: object → string

✅ No warnings generated


### Scenario 4

In [6]:
# Scenario 4: CSV with mixed types, varying date formats, and partial corruption

df_processed_mixed, metadata_mixed = process_gym_data(csv_path_mixed_corrupt)

print("\n=== SCENARIO 4: Mixed Types & Corruption ===")
print(f"\n📊 DataFrame Shape: {df_processed_mixed.shape[0]} rows × {df_processed_mixed.shape[1]} columns")
print(f"\n🏷️  Columns Identified: {list(df_processed_mixed.columns)}")
print(f"\n🎯 Column Roles: {metadata_mixed.column_roles}")
print(f"\n📁 File Type: {metadata_mixed.file_type}")
print(f"\n📈 Orientation: {metadata_mixed.orientation}")

print("\n--- Sample Data ---")
print(df_processed_mixed.head())

print("\n--- Data Types ---")
for col, dtype in metadata_mixed.inferred_dtypes.items():
    original = metadata_mixed.original_dtypes.get(col, 'unknown')
    print(f"  {col}: {original} → {dtype}")

if metadata_mixed.warnings:
    print(f"⚠️  Warnings ({len(metadata_mixed.warnings)}):")
    for warning in metadata_mixed.warnings:
        print(f"   • {warning}")
else:
    print("\n✅ No warnings generated")


=== SCENARIO 4: Mixed Types & Corruption ===

📊 DataFrame Shape: 8 rows × 6 columns

🏷️  Columns Identified: ['OrderID', 'Customer', 'OrderDate', 'Amount', 'Status', 'DeliveryDate']

🎯 Column Roles: {'OrderID': 'Numeric metric', 'Customer': 'Categorical dimension', 'OrderDate': 'Other', 'Amount': 'Numeric metric', 'Status': 'Categorical dimension', 'DeliveryDate': 'Other'}

📁 File Type: csv

📈 Orientation: long

--- Sample Data ---
   OrderID     Customer   OrderDate  Amount     Status DeliveryDate
0     1001     John Doe  2023-01-01  150.75  Completed   2023-01-05
1     1002   Jane Smith  2023-01-02    <NA>    Pending   2023-01-06
2     1003  Peter Jones  2023-03-03   200.0  Completed   2023-03-07
3     1004  Alice Brown  2023-04-04    75.2  Cancelled          NaN
4     1005    Bob White  2023-05-05   120.0  Completed   2023-05-09

--- Data Types ---
  OrderID: int64 → Int64
  Customer: object → string
  OrderDate: object → string
  Amount: object → Float64
  Status: object → categor